In [1]:
from pyspark.sql import SparkSession

catalog_name = "rainbow-data-production-iceberg"
project_id = "rainbow-data-production-483609"
spark = (
    SparkSession.builder.appName("Spark-Iceberg")
    .config("spark.master", "local[*]")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    .config("spark.driver.memory", "2G")
    .config("spark.executor.memory", "2G")
    .config("spark.driver.userClassPathFirst", "false")
    .config("spark.executor.userClassPathFirst", "false")
    .config(
        f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(f"spark.sql.catalog.{catalog_name}.type", "rest")
    .config(
        f"spark.sql.catalog.{catalog_name}.uri",
        "https://biglake.googleapis.com/iceberg/v1/restcatalog",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.warehouse",
        f"bq://projects/{project_id}",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.header.x-goog-user-project",
        project_id,
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.rest.auth.type",
        "org.apache.iceberg.gcp.auth.GoogleAuthManager",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.io-impl",
        "org.apache.iceberg.gcp.gcs.GCSFileIO",
    )
    .config(f"spark.sql.catalog.{catalog_name}.rest-metrics-reporting-enabled", "false")
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .config("spark.sql.defaultCatalog", catalog_name)
    .config(
        "spark.jars.packages",
        ",".join(
            [
                "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.10.1",
                "org.apache.iceberg:iceberg-gcp-bundle:1.10.1",
                # "com.google.auth:google-auth-library-oauth2-http:1.41.0",
                # "com.google.auth:google-auth-library-credentials:1.41.0",
                # "com.google.guava:guava:32.1.2-jre",
                # "com.google.cloud:google-cloud-storage:2.61.0",
                # "com.google.cloud:libraries-bom:26.73.0",
            ]
        ),
    )
    .getOrCreate()
)


26/01/14 00:07:33 WARN Utils: Your hostname, Trans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
26/01/14 00:07:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/tranngoctuan0910/.ivy2/cache
The jars for the packages stored in: /Users/tranngoctuan0910/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-gcp-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d91ac50-dd4b-455a-ac9f-679f671d87a4;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/tranngoctuan0910/Workspaces/Test/TestIceberg/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.1 in central
	found org.apache.iceberg#iceberg-gcp-bundle;1.10.1 in central
:: resolution report :: resolve 90ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.iceberg#iceberg-gcp-bundle;1.10.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-0d91ac50-dd4b-455a-ac9f-679f671d87a4
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/3ms)
26/01/14 00:07:33 WARN NativeCodeLoader: Un

In [2]:
spark.sql("SHOW CATALOGS").show()

+--------------------+
|             catalog|
+--------------------+
|rainbow-data-prod...|
|       spark_catalog|
+--------------------+



In [3]:
spark.sql("SELECT current_catalog();").show()

+--------------------+
|   current_catalog()|
+--------------------+
|rainbow-data-prod...|
+--------------------+



In [12]:
spark.sql("DROP NAMESPACE IF EXISTS test_namespace_bq_connection;").show()

++
||
++
++



In [13]:
# spark.sql("CREATE NAMESPACE IF NOT EXISTS test_namespace_cv ;")
spark.sql("CREATE NAMESPACE IF NOT EXISTS test_namespace_bq_connection LOCATION 'gs://rainbow-data-production-iceberg/test_namespace_bq_connection' WITH DBPROPERTIES ('gcp-region' = 'us');")

DataFrame[]

In [4]:
spark.sql("SHOW NAMESPACES").show()

+--------------------+
|           namespace|
+--------------------+
|                test|
|      test_namespace|
|     test_namespace1|
|   test_namespace_10|
|test_namespace_bq...|
|   test_namespace_cv|
+--------------------+



In [5]:
spark.sql("USE test_namespace_bq_connection;")

DataFrame[]

In [6]:
spark.sql("SHOW TABLES").show()

+--------------------+--------------------+-----------+
|           namespace|           tableName|isTemporary|
+--------------------+--------------------+-----------+
|test_namespace_bq...|my_table_with_bq_...|      false|
+--------------------+--------------------+-----------+



In [19]:
spark.sql("""
    CREATE TABLE my_table_with_bq_connection (
        id BIGINT,
        data STRING
    )
    USING iceberg
    TBLPROPERTIES (
        'bq_connection' = 'projects/rainbow-data-production-483609/locations/us/connections/iceberg_conn'
    )
""")

DataFrame[]

In [21]:
spark.sql("SHOW TABLES").show()

+--------------------+--------------------+-----------+
|           namespace|           tableName|isTemporary|
+--------------------+--------------------+-----------+
|test_namespace_bq...|my_table_with_bq_...|      false|
+--------------------+--------------------+-----------+



In [22]:
spark.sql("""
INSERT INTO my_table_with_bq_connection VALUES
  (1, 'first'), (2, 'second'), (3, 'third')
""")

DataFrame[]

In [7]:
spark.sql("SELECT * FROM my_table_with_bq_connection").show()

+---+------+
| id|  data|
+---+------+
|  1| first|
|  4|fourth|
|  5| fifth|
|  2|second|
|  6| sixth|
|  3| third|
+---+------+



In [24]:
spark.sql("""
INSERT INTO my_table_with_bq_connection VALUES
  (4, 'fourth'), (5, 'fifth'), (6, 'sixth');
""")

DataFrame[]

In [25]:
spark.sql("SELECT * FROM my_table_with_bq_connection;").show()

+---+------+
| id|  data|
+---+------+
|  4|fourth|
|  5| fifth|
|  6| sixth|
|  1| first|
|  2|second|
|  3| third|
+---+------+



In [8]:
spark.sql("""
INSERT INTO my_table_with_bq_connection VALUES
  (10, 'ten'), (11, 'eleven'), (12, 'twelve');
""")

DataFrame[]

In [12]:
spark.sql("SELECT * FROM my_table_with_bq_connection;").show()


+---+------------+
| id|        data|
+---+------------+
| 10|         ten|
| 11|      eleven|
| 12|      twelve|
|  4|      fourth|
|  1|       first|
|  5|       fifth|
|  2|      second|
|  6|       sixth|
|  3|       third|
|  7|    new_data|
|  8|   more_data|
|  7|    new_data|
|  8|   more_data|
| 23|twenty three|
| 24| twenty four|
| 29| twenty nine|
| 32|  thirty two|
| 22|  twenty two|
| 25| twenty five|
| 28|twenty eight|
+---+------------+
only showing top 20 rows

